In [ ]:
import os
import sys
from google.colab import drive

In [ ]:
#%tensorflow_version 1.x
!pip install keras-vggface
!pip install scikit-image
!pip install pydot

In [ ]:
!pip install keras_applications

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/00 thesis/data set/old/01 unzip file/thesis data kawser/00000000000'

In [ ]:
filepath = '/content/drive/MyDrive/00 thesis/data set/old/01 unzip file/thesis data kawser/epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.hdf5'

In [ ]:
os.chdir(path)

In [ ]:
ls

AK/  BCC/  BKL/  DF/  MEL/  NV/  SCC/  VASC/


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import *
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
import skimage
from skimage.transform import rescale, resize

import pydot
import time

In [ ]:
print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.3


In [ ]:
EPOCHS = 50
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 170
ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001
Resize_pixelsize = 250

In [ ]:
vgg_notop = VGGFace(model='resnet50', include_top=False, input_shape=(Resize_pixelsize, Resize_pixelsize, 3), pooling='avg')
last_layer = vgg_notop.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(DROPOUT_RATE)(x)
# l=0
# for layer in vgg_notop.layers:
#     print(layer,"["+str(l)+"]")
#     l=l+1
    
batch_norm_indices = [2, 6, 9, 13, 14, 18, 21, 24, 28, 31, 34, 38, 41, 45, 46, 53, 56, 60, 63, 66, 70, 73, 76, 80, 83, 87, 88, 92, 95, 98, 102, 105, 108, 112, 115, 118, 122, 125, 128, 132, 135, 138, 142, 145, 149, 150, 154, 157, 160, 164, 167, 170]
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False
# print('vgg layer 2 is trainable: ' + str(vgg_notop.layers[2].trainable))
# print('vgg layer 3 is trainable: ' + str(vgg_notop.layers[3].trainable))

out = Dense(8, activation='softmax', name='classifier')(x)

model = Model(vgg_notop.input, out)


optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
Name = "tf-SeNet50_cs230_{}".format(int(time.time()))
tensorboard= TensorBoard(log_dir='/content/drive/MyDrive/00 thesis/data set/old/01 unzip file/thesis data kawser/logs/{}'.format(Name))
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
# plot_model(model, to_file='model2.png', show_shapes=True)

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(patience=5)

<h1>Restart<h1>

In [ ]:
model.load_weights("/content/drive/MyDrive/00 thesis/data set/old/01 unzip file/thesis data kawser/epochs:013-val_accuracy:0.605.hdf5")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir=path  #this is the 000000000 folder

img_height=250
img_width=250
batch_size=16 #previously was 128
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode= 'categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

# model.fit_generator(
#     train_generator,
#     steps_per_epoch = train_generator.samples // batch_size,
#     validation_data = validation_generator, 
#     validation_steps = validation_generator.samples // batch_size,
#     epochs = nb_epochs)

Found 20269 images belonging to 8 classes.
Found 5062 images belonging to 8 classes.


In [ ]:
#train_generator  = get_datagen('/content/drive/My Drive/Facial_Expression_Recognition/train', True)
#dev_generator    = get_datagen('/content/dev')
#dev_generator  = get_datagen('/content/drive/My Drive/Facial_Expression_Recognition/test')
#train_generator  = get_datagen('/content/drive/My Drive/Facial_Expression_Recognition_TTV/train', True)
#dev_generator    = get_datagen('/content/drive/My Drive/Facial_Expression_Recognition_TTV/val')
#test_generator  = get_datagen('/content/drive/My Drive/Facial_Expression_Recognition_TTV/test')


In [ ]:
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
#model.load_weights('/content/drive/My Drive/Facial_Expression_Recognition_TTV/weight/epochs:071-val_accuracy:0.712.hdf5')

In [ ]:
model.summary()

In [ ]:
#history = model.fit(
#    x = train_generator,
#    validation_data = dev_generator, 
    #steps_per_epoch=28709// BS,
    #validation_steps=3509 // BS,
#    shuffle=True,
#    epochs=20,
#    callbacks=[rlrop, tensorboard, checkpoint]
#)
print(train_generator)

history = model.fit(
    x = train_generator,
    validation_data=validation_generator, 
    steps_per_epoch=20269 // BS,
    validation_steps=5062 // BS,
    shuffle=True,
    epochs=50,
    callbacks=[rlrop, tensorboard, checkpoint]
)


epoch_str = '-EPOCHS_' + str(EPOCHS)
test_acc = 'test_acc_%.3f' % results_test[1]
#model.save('/content/drive/My Drive/Facial_Expression_Recognition/weight/' + 'SENET50' + epoch_str + test_acc + '.h5')
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

Epoch 1/50
  3/158 [..............................] - ETA: 18:21 - loss: 12.8022 - accuracy: 0.2917

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate_generator(dev_generator, 3509 // BS)
print('dev loss, dev acc:', results_dev)

In [ ]:
print('\n# Evaluate on test data')
results_test = model.evaluate_generator(test_generator, 3509 // BS)
print('test loss, test acc:', results_test)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()